In [11]:
#import data
import csv
import numpy as np
data = []
with open('clusters.txt') as csvfile:
    points = csv.reader(csvfile,delimiter=',')
    for point_str in points:
        data.append([float(point_str[0]),float(point_str[1])])

In [12]:
#kmeans result
kmeans_result = []
with open('kmeans_result') as csvfile:
    points = csv.reader(csvfile,delimiter=',')
    for point_str in points:
        kmeans_result.append([float(point_str[0]),float(point_str[1]),float(point_str[2])])
kmeans_result=np.array(kmeans_result)

In [13]:
#recentralize mean to (0,0)

x_mean=np.mean(np.array(kmeans_result)[:,0])
y_mean=np.mean(np.array(kmeans_result)[:,1])

kmeans_result[:,0]=kmeans_result[:,0] - x_mean
kmeans_result[:,1]=kmeans_result[:,1] - y_mean

data = np.array(data)
data[:,0]=data[:,0]-x_mean
data[:,1]=data[:,1]-y_mean

In [14]:
#initialize clusters according to kmeans result
cluster0=[]
cluster1=[]
cluster2=[]
for record in kmeans_result:
    if record[2] == 0:
        cluster0.append(record[0:2])
    if record[2] == 1:
        cluster1.append(record[0:2])
    if record[2] == 2:
        cluster2.append(record[0:2])

#covert list to array object
cluster0 = np.array(cluster0)
cluster1 = np.array(cluster1)
cluster2 = np.array(cluster2)

#get initial gaussian parameters

#calculate mean_of each cluster
mean_cluster0 = np.mean(cluster0,axis=0)
mean_cluster1 = np.mean(cluster1,axis=0)
mean_cluster2 = np.mean(cluster2,axis=0)
#calculate covariance_of each cluster
cov_cluster0 = np.cov(cluster0.T)
cov_cluster1 = np.cov(cluster1.T)
cov_cluster2 = np.cov(cluster2.T)

In [15]:
#calculate gaussian model
def gaussian_model(mean,cov,x):
    return np.linalg.det(cov)**(-0.5)*np.exp(-0.5*np.array(x-mean).dot(np.linalg.inv(cov)).dot(np.array(x-mean).T))

In [16]:
#initial rc:N*3 metrix
initial_rc=[]
for x in data:
    temp=[]
    denominator=gaussian_model(mean_cluster0,cov_cluster0,x)+gaussian_model(mean_cluster1,cov_cluster1,x)+gaussian_model(mean_cluster2,cov_cluster2,x)
    temp.append(gaussian_model(mean_cluster0,cov_cluster0,x)/ denominator)
    temp.append(gaussian_model(mean_cluster1,cov_cluster1,x)/ denominator)
    temp.append(gaussian_model(mean_cluster2,cov_cluster2,x)/ denominator)
    initial_rc.append(temp)
initial_rc = np.array(initial_rc)
initial_rc

array([[  5.83267996e-06,   3.72523157e-11,   9.99994167e-01],
       [  1.86745846e-06,   9.82433392e-12,   9.99998133e-01],
       [  8.49362059e-03,   1.55791441e-06,   9.91504822e-01],
       [  1.58873987e-06,   1.60374443e-11,   9.99998411e-01],
       [  1.97138457e-02,   1.52287269e-06,   9.80284631e-01],
       [  9.87551213e-01,   1.18394720e-02,   6.09315478e-04],
       [  1.87301515e-01,   2.30516897e-04,   8.12467968e-01],
       [  9.63330331e-01,   2.94623820e-02,   7.20728708e-03],
       [  7.77306323e-03,   1.51277752e-07,   9.92226785e-01],
       [  9.80396532e-01,   1.93938329e-02,   2.09634911e-04],
       [  6.04212783e-02,   2.13717738e-05,   9.39557350e-01],
       [  9.31381242e-01,   7.72107253e-03,   6.08976859e-02],
       [  9.67672106e-01,   3.23278881e-02,   5.84828980e-09],
       [  9.85003301e-01,   1.45898792e-02,   4.06820116e-04],
       [  1.68385819e-02,   1.03442374e-05,   9.83151074e-01],
       [  9.83984050e-01,   2.51848875e-04,   1.5764101

In [33]:
#EM algorithm
#We put M step first because we have done initial E-step outside the loop
def EM(initial_rc,iterations=100):
    rc=initial_rc
    diff = 100
    old = [-1]
    while (iterations and diff > 1):
        print "iteration:"+ str(iterations)
        #M step
        #calculate weighted data of each cluster(component) according to rc
        c0_data=[]
        c1_data=[]
        c2_data=[]
        for index,x in enumerate(data):
            c0_data.append(np.array(x)*rc[:,0][index])
            c1_data.append(np.array(x)*rc[:,1][index])
            c2_data.append(np.array(x)*rc[:,2][index])
        c0_data=np.array(c0_data)
        c1_data=np.array(c1_data)
        c2_data=np.array(c2_data)
        mean_cluster0=np.mean(c0_data,axis=0)
        mean_cluster1=np.mean(c1_data,axis=0)
        mean_cluster2=np.mean(c2_data,axis=0)
        cov_cluster0=np.cov(c0_data.T)
        cov_cluster1=np.cov(c1_data.T)
        cov_cluster2=np.cov(c2_data.T)
        print cov_cluster2
        
        #calculate loglikelyhood
        a = np.sum(rc,axis=0)/np.sum(rc)
        loglh=0
        for x in np.array(data):
            log_temp=a[0]*gaussian_model(mean_cluster0,cov_cluster0,x)+a[1]*gaussian_model(mean_cluster1,cov_cluster1,x)+a[2]*gaussian_model(mean_cluster2,cov_cluster2,x)
            loglh+=np.log(log_temp)
        print "log likelyhood: "+str(loglh)
        diff = abs(old.pop()-loglh)
        old.append(loglh)
        print "log likelyhood diff : "+str(diff)
        iterations=iterations-1
        
        #E step
        
        #update rc
        rc=[]
        for x in np.array(data):
            temp=[]
            denominator=gaussian_model(mean_cluster0,cov_cluster0,x)+gaussian_model(mean_cluster1,cov_cluster1,x)+gaussian_model(mean_cluster2,cov_cluster2,x)
            temp.append(gaussian_model(mean_cluster0,cov_cluster0,x)/ denominator)
            temp.append(gaussian_model(mean_cluster1,cov_cluster1,x)/ denominator)
            temp.append(gaussian_model(mean_cluster2,cov_cluster2,x)/ denominator)
            rc.append(temp)
        rc = np.array(rc)
        